In [1]:
import pandas as pd
import geopy as gp
from geopy import distance
import re
import datetime

In [5]:
# change path back to original file name; changed out of laziness
df = pd.read_excel("schnitzler_places.xlsx")


TypeError: read_excel() got an unexpected keyword argument 'columns'

In [4]:
df

,places_per_day_with-pmb,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,date,cert,place,geonames,lat,long,pmb,akon,note1,note2,note3,note4,note5
1,1869-01-01,,Thalhof,NaN,47.70466,15.84704,https://pmb.acdh.oeaw.ac.at/apis/entities/enti...,NaN,,,,,
2,1869-01-02,,Thalhof,NaN,47.70466,15.84704,https://pmb.acdh.oeaw.ac.at/apis/entities/enti...,NaN,,,,,
3,1879-01-01,,Wien,https://sws.geonames.org/2761369/,48.20849,16.37208,https://pmb.acdh.oeaw.ac.at/apis/entities/enti...,https://iiif.onb.ac.at/images/AKON/AK117_016/0...,NaN,,,,
4,1879-01-02,,Wien,https://sws.geonames.org/2761369/,48.20849,16.37208,https://pmb.acdh.oeaw.ac.at/apis/entities/enti...,https://iiif.onb.ac.at/images/AKON/AK117_016/0...,NaN,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24288,1931-10-17,,Wien,https://sws.geonames.org/2761369/,48.20849,16.37208,https://pmb.acdh.oeaw.ac.at/apis/entities/enti...,https://iiif.onb.ac.at/images/AKON/AK117_016/0...,NaN,,,,
24289,1931-10-18,,Wien,https://sws.geonames.org/2761369/,48.20849,16.37208,https://pmb.acdh.oeaw.ac.at/apis/entities/enti...,https://iiif.onb.ac.at/images/AKON/AK117_016/0...,NaN,,,,
24290,1931-10-19,,Wien,https://sws.geonames.org/2761369/,48.20849,16.37208,https://pmb.acdh.oeaw.ac.at/apis/entities/enti...,https://iiif.onb.ac.at/images/AKON/AK117_016/0...,NaN,,,,
24291,1931-10-20,,Wien,https://sws.geonames.org/2761369/,48.20849,16.37208,https://pmb.acdh.oeaw.ac.at/apis/entities/enti...,https://iiif.onb.ac.at/images/AKON/AK117_016/0...,NaN,,,,


In [3]:
test = ["Theater" in el for el in df.date]

AttributeError: 'DataFrame' object has no attribute 'date'

In [ ]:
# drop the rows with theater as date
df = df[[ not el for el in test]]

In [ ]:
df["date_date"] = pd.to_datetime(df.date)

In [ ]:
def iterate_dataframe(dataframe, distance_per_day):
    """
    distance_per_day (int): give a number that represents the max distance (in kilometer "luftlinie") 
    between two geogpraphical points. Used to calculate a threshold to test the calculated distance per day against.
    dataframe: a pandas DataFrame.
    """
    df = dataframe
    res = []
    
    def get_time_sep(date1, date2):
        """
        helper function to calculate the distance between two timestamps in days.
        """
        return (date1 - date2).days

    def get_distance(a, b):
        """
        helper function to calculate the distance between two geographical points in km
        """
        return int(gp.distance.distance(a,b).km)

    for idx, el in df.iterrows():
        if len(df) >= idx >= 1:
            last = df.iloc[idx-1]
            
            # Try-Except needed because there are malformed floats (strings) in the lat-long columns
            try: 
                
                # days between a given row and the row before
                time_delta_in_days = get_time_sep(el.date_date, last.date_date)
                # distance in km between the lat-long coordinates in one row and the row before
                distance_in_km = get_distance((el.lat, el.long), (last.lat, last.long))
                
                # tolerated_time_distance: f.e.: if you allow 300km per day, a distance of 600km for a two day-trip is accepted.
                # abs is used here, because some timedeltas are negative (as we check against the entry before)
                tolerated_time_distance = abs(time_delta_in_days * distance_per_day)
                
                # some timedelta where 0 and produce a division by zero error, that is caught here
                if time_delta_in_days != 0:
                    # abs is used here, because some timedeltas are negative (as we check against the entry before)
                    calculated_time_distance = abs(int(distance_in_km / time_delta_in_days))
                    
                    # the juicy part: if the calculated distance is bigger then the threshold, do your thing:
                    if tolerated_time_distance < calculated_time_distance and el.place != last.place:
                        #print(el.date, el.place, last.date, last.place)
                        #print(idx, idx-1, "tolerated_time_distance", tolerated_time_distance, "km per tag: ", calculated_time_distance, "tage:", time_delta_in_days, "distance:", distance_in_km)
                        res.append({"idx_from":idx-1,"from":last.place, "idx_to":idx, "to":el.place, "km_per_day":calculated_time_distance, "threshold":distance_per_day, "date_from":last.date, "date_to":el.date})
            except Exception as e:
                # Print the malformed lat-long columns and all other occuring errors with the row on which the script fails.
                print(f"Error in row: {idx}. Message: {e}")
                continue
    return res

In [ ]:
res = iterate_dataframe(df, 300)

In [ ]:
tf = pd.DataFrame(res)

In [ ]:
tf

In [ ]:
tf.to_csv("output_schnitzler_places.csv", index=False, encoding="utf-8")

In [ ]:
# I used this to test for string-junk in the date column
for el in df.date:
    test = re.search(r"[A-Za-z]", el)
    if test:
        print(test.group(0), el)